In [1]:
import tweepy

API_KEY = "HZh8MFUydAVaIN50fYh2w5iVr"
API_SECRET = "BjkcY9H2tZhTHH2gDiyeByRJaBWExXxW523ZXvhy5jyh0mTlLn"
ACCESS_TOKEN = "301590896-xTlLyxVVYURtMRFDyhBa4BHEOEAHheCa9iEdicQv"
ACCESS_TOKEN_SECRET = "90soM5pLMXfuWkP1BIGVEDZFrBWkearvAiEeJ72JbW5rc"

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
def tweetSearch(query, limit = 10000, language = "fr", date_since="2019-12-05"):
    text = ""
    for tweet in tweepy.Cursor(api.search, q=query, lang=language, since=date_since).items(limit):
        text += tweet.text.lower()
    return text

In [ ]:
greve = tweetSearch("grève")